# <center>Binary Search

## [35. Search Insert Position (easy)](https://leetcode-cn.com/problems/search-insert-position/)

Given a sorted array and a target value, return the index if the target is found. If not, return the index where it would be if it were inserted in order.

You may assume no duplicates in the array.

<font color='dd0000'>

二分法，如果不存在要找到插入位置，即去搜索**比```target```大的第一个位置**。如果所有数都比```target```小，就插在最后一个位置 + 1

</font>

<font color='dd0000'>

**二分注意事项：**

```/``` 是整除，它的行为是“向下取整”，造成了 ```int mid = (left + right) / 2``` 这种写法 ```mid``` 永远取不到待搜索区间里最右边的位置。

对```mid```被分到右边的情况，即```left = mid, right = mid - 1```这种折半写法，在待搜索区间收缩到只剩下 2 个元素的时候，就有可能造成死循环（因为```mid```一直等于```left```，```left```又一直取```mid```）

**如果遇到死循环，将求```mid```改为```int mid = left + (right - left + 1) / 2```即可**

</font>

In [1]:
public int searchInsert(int[] nums, int target) {
     int left = 0, right = nums.length - 1;
     if(target > nums[right]){ // 目标值大于数组中最大值
         return right + 1;
     }
     while(left < right){
         int mid = left + (right - left) / 2;
         if(nums[mid] == target){
             return mid;
         } else if(nums[mid] < target){
             left = mid + 1;
         } else {
             right = mid;
         }
     }
     return left;
}

## ⭐[33. Search in Rotated Sorted Array (medium)](https://leetcode-cn.com/problems/search-in-rotated-sorted-array/)

Suppose an array sorted in ascending order is rotated at some pivot unknown to you beforehand.

(i.e., ```[0,1,2,4,5,6,7]``` might become ```[4,5,6,7,0,1,2]```).

You are given a target value to search. If found in the array return its index, otherwise return -1.

You may assume no duplicate exists in the array.

Your algorithm's runtime complexity must be in the order of ```O(log n)```.

In [ ]:
// 思路一：先通过二分查找出数组中最小值的下标，即为旋转点，再根据target的大小确定去旋转点的左右进行二分查找
// 注意点：1.找旋转点的方法，如果循环中没有找到，则返回0，包含整个数组都旋转的情况（等于没有旋转）
//         2.判断该去旋转点的哪边搜索：如果旋转点值是0，说明数组没有旋转，对整个数组进行搜索；
//                                     否则，因为旋转点左边的值都大于它自己,所以如果【target < nums[0]】,说明target应在旋转点右边或不存在

 public int search(int[] nums, int target) {
    if(nums.length < 1)
            return -1;

    int left = 0;
    int right = nums.length - 1;
    int rotatedIndex = findRotatedIndex(nums, left, right); // 找旋转点
    
    if(target == nums[rotatedIndex])  // target刚好是旋转点的值
        return rotatedIndex;
    
    if(rotatedIndex == 0)     // !!!旋转点是0的情况,需特殊考虑
        return binarySearch(nums, left, right, target);
    
    if(target < nums[0])  // target比起点值小，说明在旋转点右边
        return binarySearch(nums, rotatedIndex + 1, right, target);      
    
    return binarySearch(nums, left, rotatedIndex - 1, target);   // else，在旋转点左边    
}

// 找旋转点，旋转点一定比它前面的值小
private int findRotatedIndex(int[] nums, int left, int right){
    if(nums[left] <= nums[right])  // 起始值<=终点值，说明旋转点在0
        return 0;
        
    while(left <= right){
        int mid = left + (right - left) / 2;
        if(nums[mid] > nums[mid + 1]){
            return mid + 1;
        }
        if(nums[left] <= nums[mid]){ // 说明左边有序，旋转点在右边
            left = mid + 1;
        } else {
            right = mid - 1;
        }
    }
    return 0;
}

// 单纯的二分查找
private int binarySearch(int[] nums, int left, int right, int target){
    while(left <= right){
        int mid = left + (right - left) / 2;
        if(target == nums[mid])
            return mid;
        else if(target < nums[mid]){
            right = mid - 1;
        } else {
            left = mid + 1;
        }
    }
    return -1;
}

In [ ]:
// 思路二：将数组一分为二，其中一定有一个是有序的，另一个可能是有序，也能是部分有序。此时有序部分用二分法查找
//         无序部分再一分为二，其中一个一定有序，另一个可能有序，可能无序。就这样循环.

public int search2(int[] nums, int target) {
    return helper(nums, 0, nums.length - 1, target);
}

private int helper(int[] nums, int left, int right, int target){
    if(left > right){
        return -1;
    }
    int mid = left + (right - left) / 2;
    int res = -1;
    if(nums[left] <= nums[mid]){  // left < mid 说明左边部分是有序的
        res = binarySearch(nums, left, mid, target);
        if(res != -1){
            return res; // 对左半二分查找，如果找到直接返回
        } else {        // 如果没有找到，则递归对右半部分进行操作
            return helper(nums, mid + 1, right, target);
        }
    } else {
        res = binarySearch(nums, mid + 1, right, target);
        if(res != -1){
            return res;
        } else {
            return helper(nums, left, mid, target);
        }
    }
}

private int binarySearch(int[] nums, int left, int right, int target){       
    while(left <= right){
        int mid = left + (right - left) / 2;
        if(target < nums[mid]){
            right = mid - 1;
        } else if(target > nums[mid]){
            left = mid + 1;
        } else{
            return mid;
        }
    }
    return -1;
}

// 思路类似，不用递归，更简洁
public int search3(int[] nums, int target) {
    int left = 0;
    int right = nums.length - 1;
    int mid = left + (right - left) / 2;
    while(left <= right){
        if(target == nums[mid])
            return mid;
            
        // 用二分划分出有序的部分，通过有序的部分来判断target位于数组的哪个部分
        if(nums[left] <= nums[mid]){ // 左边有序
            if(target >= nums[left] && target <= nums[mid]){ // target在左边
                right = mid - 1;
            } else {   // target在右边
                left = mid + 1;
            }
        } else {  // 右边有序
            if(target >= nums[mid + 1] && target <= nums[right]){ // target在右边
                left = mid + 1;
            } else {
                right = mid - 1;
            }
        }
        mid = left + (right - left) / 2; // 更新mid
    }
    return -1;
}

## ⭐[300. Longest Increasing Subsequence (medium)](https://leetcode-cn.com/problems/longest-increasing-subsequence/)

Given an unsorted array of integers, find the length of longest increasing subsequence.

<font color='dd0000'>二分查找的解法，时间复杂度$O(nlog(n))$</font>

In [5]:
// 用一个LIS数组来始终维护一个【最长递增子序列】
// 遍历原数组时，如果值大于LIS数组的最后一个数，就添加到LIS的后面；如果小于，就在LIS数组中找到比它大的最小的数，替换掉
// 这样，LIS数组是一个有序的递增数组，所以在每次插入新值时，可以用二分查找，从而降低时间复杂度到log(n)
// LIS数组的长度就是最长递增子序列的长度（不一定是一个合法的序列，但长度一定是最长的）

public int lengthOfLIS(int[] nums) {
    int[] LIS = new int[nums.length]; // LIS数组最长为nums的长度，所以初始化为最大的情况
    int len = 0;  // 用来保存LIS的长度
    for(int i = 0; i < nums.length; i++){
        int lo = 0;
        int hi = len;  // hi设为len是因为，如果nums[i]是最大的数，可以二分查找后插在LIS[len]的位置
        while(lo < hi){
            int mid = lo + (hi - lo) / 2;  // 二分查找
            if(nums[i] > LIS[mid]){
                lo = mid + 1;
            } else {
                hi = mid;  // 注意：nums[i]小于mid时,要保留mid，因为要找的是比nums[i]大的最小的数，可能就落在mid上
            }
        }
        LIS[lo] = nums[i]; // 替换值
        if(lo == len) len++; // 如果是插在后面，就将长度len++
    }
    return len;
}

## [1095. Find in Mountain Array (hard)](https://leetcode-cn.com/problems/find-in-mountain-array/)

<font color='dd0000'>首先通过一次二分查找找到**山顶**，即将数组分成两个有序数组。再分别在这两个有序数组中查找```target```</font>

In [2]:
interface MountainArray {
    public int get(int index);
    public int length();
}

In [5]:
public int findInMountainArray(int target, MountainArray mountainArr) {
    int top = findTop(mountainArr);
    // 0 ~ top 递增数组，top ~ length - 1 递减数组
    int l = binarySearch(target, mountainArr, 0, top, target);
    if(l != -1){
        return l;
    }
    return binarySearch2(target, mountainArr, top + 1, mountainArr.length() - 1, target);
}

private int findTop(MountainArray mountainArr){ // 找山顶，即找第一个比后面的数大的那个数的下标
    int left = 0, right = mountainArr.length() - 1;
    while(left < right){
        int mid = left + (right - left) / 2;
        if(mountainArr.get(mid) > mountainArr.get(mid + 1)){
            right = mid;
        } else {
            left = mid + 1;
        }
    }
    return left;
}

private int binarySearch(int target, MountainArray mountainArr, int left, int right){ // 二分查找递增数组
    int left = 0, right = mountainArr.length() - 1;
    while(left <= right){
        int mid = left + (right - left) / 2;
        if(mountainArr.get(mid) == target){
            return mid;
        } else if(mountainArr.get(mid) < target){
            left = mid + 1;
        } else {
            right = mid - 1;
        }
    }
    return -1;
}

private int binarySearch2(int target, MountainArray mountainArr, int left, int right){
    int left = 0, right = mountainArr.length() - 1;
    while(left <= right){
        int mid = left + (right - left) / 2;
        if(mountainArr.get(mid) == target){
            return mid;
        } else if(mountainArr.get(mid) < target){
            right = mid - 1;
        } else {
            left = mid + 1;
        }
    }
    return -1;
}

## ⭐[4. Median of Two Sorted Arrays (hard)](https://leetcode-cn.com/problems/median-of-two-sorted-arrays/)

There are two sorted arrays **nums1** and **nums2** of size m and n respectively.

Find the median of the two sorted arrays. The overall run time complexity should be **O(log (m+n))**.

You may assume **nums1** and **nums2** cannot be both empty.

[<font color='dd00dd'>题 解</font>](https://leetcode-cn.com/problems/median-of-two-sorted-arrays/solution/xiang-xi-tong-su-de-si-lu-fen-xi-duo-jie-fa-by-w-2/)

<font color='dd0000'>

**重点：**$O(log(m+n))$ 的复杂度

**方法一：** 将求中位数转为求第 $k$ 小的数

</font>

In [ ]:
 // 转换为求第k个数
public double findMedianSortedArrays(int[] nums1, int[] nums2) {
    int n1 = nums1.length, n2 = nums2.length;
    int n = n1 + n2;
    int k1 = (n + 1) / 2, k2 = (n + 2) / 2;
    // 对奇数和偶数都统一求第 (n+1)/2 和第 (n+2)/2 个数再取平均，对奇数而言两次求到的是同一个值
    return 0.5 * (findKth(nums1, 0, n1 - 1, nums2, 0, n2 - 1, k1) + findKth(nums1, 0, n1 - 1, nums2, 0, n2 - 1, k2));
}

private int findKth(int[] nums1, int left1, int right1, int[] nums2, int left2, int right2, int k){
    int len1 = right1 - left1 + 1, len2 = right2 - left2 + 1;
    
    if(len1 > len2){ // 始终将nums1设为短的数组
        return findKth(nums2, left2, right2, nums1, left1, right1, k);
    }
        
    if(left1 > right1){ // 相当于nums1全部都被排除了
        return nums2[left2 + k - 1];
    }
    if(k == 1){  //求第一个数，就直接取两个数组当前范围中最小的数
        return Math.min(nums1[left1], nums2[left2]);
    }

    int i = left1 + Math.min(len1, k / 2) - 1; // 两个数组分别取 k/2 个数，考虑长度不够 k/2 的情况
    int j = left2 + Math.min(len2, k / 2) - 1; // i和j 表示下标
    
    if(nums1[i] < nums2[j]){ // 如果第一个数组的第 k/2 个数更小，就去掉这 k/2 个数，范围缩减为 i+1 ~ right1，k 同时缩减
        return findKth(nums1, i + 1, right1, nums2, left2, right2, k - (i - left1 + 1));
    } else{
        return findKth(nums1, left1, right1, nums2, j + 1, right2, k - (j - left2 + 1));
    }
}

<font color='dd0000'>**方法二：** 二分查找分割边界</font>

In [ ]:
// 【重点！！】：各种边界情况
public double findMedianSortedArrays(int[] nums1, int[] nums2) {
    int n1 = nums1.length, n2 = nums2.length;
    int n = n1 + n2;
    int k1 = (n + 1) / 2, k2 = (n + 2) / 2;
    // 同样将奇数和偶数的情况统一
    return 0.5 * (binarySearch(nums1, nums2, k1) + binarySearch(nums1, nums2, k2));
}

private int binarySearch(int[] nums1, int[] nums2, int k){
    int len1 = nums1.length, len2 = nums2.length;
    if(len1 > len2){
        return binarySearch(nums2, nums1, k);
    }
        
    int left = 0, right = len1 - 1;
    while(left <= right){
        int i = left + (right - left) / 2; // nums1分界线下标
        int j = k - (i + 1) - 1;   // nums2分界线下标
        // 求左半部分最大值, 考虑i, j的边界情况
        int max = 0;
        if(j < 0){  // 因为i是由left和right确定的,所以i的取值一定是有效的,此时需考虑j是否小于0.且由于nums1更短,所以j不可能越右边界
            max = nums1[i];
        } else {
            max = Math.max(nums1[i], nums2[j]);
        }
        // 求右半部分最小值
        int min = 0;
        if(i >= len1 - 1 && j >= len2 - 1){ //i,j都是数组最后一个元素的情况,即右半部分没有值,那么直接返回max(两个数组都只有1个数的情况)
            return max;
        }
        else if(i >= len1 - 1){ // 如果其中有一个数组右边没有值
            min = nums2[j + 1];
        } else if(j >= len2 - 1){
            min = nums1[i + 1];
        } else {     // 两个数组两边都有值
            min = Math.min(nums1[i + 1], nums2[j + 1]);
        }
        // 比较
        if(max <= min){ // 找到解了，返回
            return max;
        } else {
            if(j >= 0 && nums1[i] > nums2[j] || j < 0){ // !!! j可能小于0，需考虑到
                right = i - 1;
            } else {
                left = i + 1;
            }
        }
    }
    // 没有查找到，说明可能nums1全部都在前 k 个数中(都很小) 或者 全部都不在前 k 个数中(都很大)
    if(left > len1 - 1){
        return nums2[k - len1 - 1];
    } else {
        return nums2[k - 1];
    }
}

## [287. Find the Duplicate Number (medium)](https://leetcode-cn.com/problems/find-the-duplicate-number/)

Given an array nums containing n + 1 integers where each integer is between 1 and n (inclusive,即包括1和n), prove that at least one duplicate number must exist. Assume that there is only one duplicate number, find the duplicate one.

**Note:**

1. You must **not modify** the array (assume the array is read only).
2. You must use only constant, O(1) extra space.
3. Your runtime complexity should be less than O(n2).
4. There is only one duplicate number in the array, but it could be repeated more than once.

<font color='dd0000'>

**方法一：**值的二分查找。虽然数组无序，但是限定了数的范围都在【1~n】,且【**包括1和n**】。所以可以先**猜一个数，比如中间数**```mid```，然后遍历数组，统计**小于等于**这个数的个数```count```，如果这个个数```count```**大于**这个数，那么重复数就一定在```[left,mid]```中，从而将搜索范围减小一半。时间复杂度$O(nlog(n))$

</font>

In [1]:
public int findDuplicate(int[] nums) {
    int n = nums.length;   // 数组长度为n，则数的范围为1 ~ n-1
    int left = 1, right = n - 1;  // left和right都是数值，不是下标
    while(left < right){
        int mid = left + (right - left) / 2;  // 注意比较的是数值 不是下标
        int count = 0;
        for(int num : nums){  // 统计小于等于mid的个数
            if(num <= mid){
                count++;
            }
        }
        if(count > mid){
            right = mid;
        } else {
            left = mid + 1;
        }
    }
    return left;
}

<font color='dd0000'>**方法二：**快慢指针。画一条路径（类似链表），将数组的值看成是下一个数的下标，由于有重复的数，所以这条路径必然有环，而入环点就是那个重复数，用快慢指针找到入环点即可。</font>

In [2]:
public int findDuplicate(int[] nums) {
    int slow = nums[0], fast = nums[0];  // 起点
    while(true){
        slow = nums[slow];
        fast = nums[fast];
        fast = nums[fast];
        if(slow == fast){  // 相遇之后，将一个指针放到起点，再同步前进，再次相遇的点就是入环点
            fast = nums[0];
            while(slow != fast){
                slow = nums[slow];
                fast = nums[fast];
            }
            return slow;
        }
    }
}

## [475. 供暖器 (easy)](https://leetcode-cn.com/problems/heaters/)

冬季已经来临。 你的任务是设计一个有固定加热半径的供暖器向所有房屋供暖。

现在，给出位于一条水平线上的房屋和供暖器的位置，找到可以覆盖所有房屋的最小加热半径。

所以，你的输入将会是房屋和供暖器的位置。你将输出供暖器的最小加热半径。

**说明:**

1. 给出的房屋和供暖器的数目是非负数且不会超过 25000。
2. 给出的房屋和供暖器的位置均是非负数且不会超过10^9。
3. 只要房屋位于供暖器的半径内(包括在边缘上)，它就可以得到供暖。
4. 所有供暖器都遵循你的半径标准，加热的半径也一样。

<font color='dd0000'>

**核心思路：** 对每个房屋，找和它**最近的**供暖器，所有这些最近距离中的**最大值**就是结果

方法一：双指针。对每个房屋，两个指针指向和它最近的两个供暖器(一左一右或两个都在右边)，其中一定有一个是最近的，不断取最大

方法二：对供暖器的位置排序。对每个房屋，用二分搜索距离它最近的供暖器，不断取最大

</font>

In [3]:
// 解法一：双指针
public int findRadius(int[] houses, int[] heaters) {
    Arrays.sort(heaters);
    Arrays.sort(houses);
    int res = 0;
    if(heaters.length == 1){  // 特殊情况，只有一个供暖器时两个指针会越界，所以单独考虑
        int pos = heaters[0];
        for(int house : houses){
            res = Math.max(res, Math.abs(house - pos));
        }
        return res;
    }
    int i = 0, j = 1;
    for(int house : houses){
        while(j < heaters.length - 1 && heaters[j] < house){ // 相当于让j每次指向当前房屋右边的第一个供暖器
            i++;                                             // 无论i是在左还是在右，i j中必有一个最近的
            j++;
        }
        int x = Math.abs(house - heaters[i]);
        int y = Math.abs(house - heaters[j]);
        res = Math.max(res, Math.min(x, y));
    }
    return res;
}

In [4]:
// 解法二：二分搜索
public int findRadius2(int[] houses, int[] heaters) {
    Arrays.sort(heaters);  // 搜索供暖器，所以先对供暖器排序
    for(int house : houses){
        int left = 0, right = heaters.length - 1;
        while(left < right){
            int mid = left + (right - left) / 2;
            if(heaters[mid] >= house){  // 当大于时保留mid，所以搜索的是比house大的第一个位置
                right = mid;
            } else {
                left = mid + 1;
            }
        }
        // 退出循环时，left(=right)可能正好是house的位置，可能是house右边第一个位置，可能是house左边第一个位置(所有供暖器都小于house)
        // 而当left是house右边第一个位置时，且它前面还有供暖器(即left != 0)时，可能前一个位置更近，需要比较
        // 而其他情况(left == 0 || left在house右边且前面没有供暖器 || left在house左边)，只用计算它们之间的距离即可
        if(heaters[left] > house && left > 0){
            int pre = house - heaters[left - 1];
            int post = heaters[left] - house;
            res = Math.max(res, Math.min(pre, post));
        } else {
            res = Math.max(res, Math.abs(house - heaters[left]));
        }
    }
    return res;
}

## [1300. 转变数组后最接近目标值的数组和 (medium)](https://leetcode-cn.com/problems/sum-of-mutated-array-closest-to-target/)

给你一个整数数组 ```arr``` 和一个目标值 ```target``` ，请你返回一个整数 ```value``` ，使得将数组中所有大于 ```value``` 的值变成 ```value``` 后，数组的和最接近  ```target``` （最接近表示两者之差的绝对值最小）。

如果有多种使得和最接近 ```target``` 的方案，请你返回这些整数中的最小值。

请注意，答案不一定是 ```arr``` 中的数字。

<font color='dd0000'>

**思路一：**枚举+二分

**思路二：**数学计算

**核心：**随着取值```value```的增大，数组和是**递增**的

</font>

In [1]:
// 思路一：枚举+二分

// value的取值范围为[0,max],先枚举一个value,再在数组中搜索value附近的值,计算和sum
// 枚举value可以【线性枚举】,也可以【二分枚举】加快速度
// 因为sum是递增的
// 所以线性枚举时,当sum > target且sum - target > diff时即可
// 二分枚举时,每次根据sum和target的大小关系调整value的值
public int findBestValue(int[] arr, int target) {
    int n = arr.length;
    Arrays.sort(arr);
    int max = arr[n - 1];
    // 为了计算数组和,先求出数组的前缀和
    int[] prefix = new int[n + 1]; // prefix[i]表示0~i-1位的和
    for(int i = 1; i <= n; i++){
        prefix[i] = prefix[i - 1] + arr[i - 1];
    }
    // 二分枚举
    int lo = 0, hi = max;
    while(lo < hi){
        int value = lo + (hi - lo) / 2;
        // 再在数组中二分搜索【比value大的第一个数】
        int left = 0, right = n - 1;
        while(left < right){
            int mid = left + (right - left) / 2;
            if(arr[mid] > value){
                right = mid;
            } else {
                left = mid + 1;
            }
        }
        int sum = prefix[left] + (n - left) * value; // 当前value下的数组和
        if(sum >= target){ // 搜索的是【使sum比target大的第一个value】
            hi = value;
        } else{
            lo = value + 1;
        } 
    }
    // 因为要求最小的value,所以搜索到的value可能偏大,要判断一下value和value-1谁更接近
    int sum1 = getSum(arr, prefix, lo - 1);
    int sum2 = getSum(arr, prefix, lo);
    if((target - sum1) <= (sum2 - target)){
        return lo - 1;
    } else{
        return lo;
    }
}

private int getSum(int[] arr, int[] prefix, int value){
    int left = 0, right = arr.length - 1;
    while(left < right){
        int mid = left + (right - left) / 2;
        if(arr[mid] > value){
            right = mid;
        } else {
            left = mid + 1;
        }
    }
    int sum = prefix[left] + (arr.length - left) * value;
    return sum;
}

    //线性枚举
    /*
    int diff = target;
    int res = 0;
    for(int value = 0; value <= max; value++){
        int left = 0, right = n - 1;
        while(left < right){
            int mid = left + (right - left) / 2;
            if(arr[mid] <= value){
                left = mid + 1;
            } else {
                right = mid;
            }
        }
        int sum = 0;
        if(left == 0){
            sum = n * value;
        } else {
            sum = prefix[left] + (n - left) * value;
        }
        if(sum > target && sum - target > diff){ //当sum大于target且差值大于最小差值时就不必再枚举后面的数
            break;
        }
        if(Math.abs(target - sum) < diff){
            res = value;
            diff = target - sum;
        }
    }
    return res;
    */

In [ ]:
// 思路二：数学计算
//

public int findBestValue(int[] arr, int target) {
    Arrays.sort(arr);
    // 首先求前缀和
    int[] prefix = new int[arr.length + 1];
    for(int i = 1; i < prefix.length; i++){
        prefix[i] = prefix[i - 1] + arr[i - 1];
    }
    for(int i = 0; i < arr.length; i++){ // 一个数一个数的看
        int prefixSum = prefix[i];
        int aveInt = (target - prefixSum) / (arr.length - i); // 计算以当前i,将剩余的值全都取相同值时的平均值且向下取整
        if(aveInt > arr[i]){ // 如果剩余整型平均值大于当前值,说明arr[i]太小,剩余数全部变为arr[i]不够,应再增大
            continue;
        }else{ // 当前值比剩余整型平均值第一次更大时,说明剩余值全部变为arr[i]后最接近target
            // 但还要看平均值的小数点情况取大小
            double ave = (double)(target - prefixSum) / (arr.length - i);
            if(ave - aveInt > 0.5){ // 五舍六入
                return aveInt + 1; //如果平均值小数点大于0.5,说明取aveInte+1的和更接近target
            } else {
                return aveInt;
            }
        }
    }
    return arr[arr.length - 1]; //如果当前值一直小于剩余平均值,说明只能取数组中最大的数才能使和最大以接近target
}

## [378. Kth Smallest Element in a Sorted Matrix (medium)](https://leetcode-cn.com/problems/kth-smallest-element-in-a-sorted-matrix/)

Given a n x n matrix where each of the rows and columns are sorted in ascending order, find the kth smallest element in the matrix.

Note that it is the kth smallest element in the sorted order, not the kth distinct element.

<font color='dd0000'>

方法一：堆。构造一个大小为 $k$ 的大顶堆，当容量满时，遇到更小的数就弹出，最终栈顶留下的就是第 $k$ 小的数

方法二：归并排序。因为每个子数组都是有序的，直接两两归并排序返回第 $k-1$个数即可

**方法三：** 值的二分搜索(而不是下标)。上面两个方法都没有用到列有序的条件。每次搜索一个中间值，计算比它小的个数(利用到列有序)，再缩小范围

```
在[left, right]中符合条件(count(矩阵中小于它的数)==k)的数m可能会有多个，这些数中，最小的那个(设为a)一定在矩阵中.因为对于任意整数i(i<a) ，count(i)<k，直到i等于a时，count将第一次等于k。因此二分搜索能找到第一个使count==k的位置，就一定是所求的值
```
</font>

In [3]:
// 二分搜索
public int kthSmallest(int[][] matrix, int k) {
    int n = matrix.length;
    int left = matrix[0][0]; // 最小值
    int right = matrix[n - 1][n - 1]; // 最大值
    while(left < right){
        int m = left + (right - left) / 2; // 一个中间值
        if(countsOfLessthanOrEqual2m(matrix, m) >= k){ // 求比m小的个数, 如果大于等于k, 保留
            right = m;
        } else {
            left = m + 1;
        }
    }  // 最终搜索到的是 有k个数小于等于m的最小的m 而这个值一定在矩阵中
    return left;
}

// 求矩阵中小于等于m的个数有多少，利用矩阵行列均有序的条件
private int countsOfLessthanOrEqual2m(int[][] matrix, int m){
    int row = 0, col = matrix.length - 1;
    int count = 0;
    while(row < matrix.length && col >= 0){
        if(matrix[row][col] <= m){
            count += col + 1;
            row++;
        } else {
            col--;
        }
    }
    return count;
}

In [ ]:
// 堆
public int kthSmallest(int[][] matrix, int k) {
    PriorityQueue<Integer> heap = new PriorityQueue<>((a, b) -> {return b - a;});
    for(int i = 0; i < matrix.length; i++){
        for(int j = 0; j < matrix[0].length; j++){
            if(heap.size() < k){
                heap.offer(matrix[i][j]);
            } else if(heap.size() == k){
                if(matrix[i][j] < heap.peek()){
                    heap.poll();
                    heap.offer(matrix[i][j]);
                }
            }
        }
    }
    return heap.peek();
}

// 归并排序
public int kthSmallest(int[][] matrix, int k) {
    int[] arr = split(matrix, 0, matrix.length - 1);
    return arr[k - 1];
}

private int[] split(int[][] matrix, int left, int right){
    if(left == right){
        return matrix[left];
    }
    int mid = left + (right - left) / 2;
    int[] l = split(matrix, left, mid);
    int[] r = split(matrix, mid + 1, right);
    return merge(l, r);
}

private int[] merge(int[] l, int[] r){
    int[] res = new int[l.length + r.length];
    int i = 0, j = 0, index = 0;;
    while(i < l.length && j < r.length){
        if(l[i] < r[j]){
            res[index++] = l[i++];
        } else if(l[i] > r[j]){
            res[index++] = r[j++];
        } else{
            res[index++] = l[i++];
            res[index++] = r[j++];
        }
    }
    while(i < l.length){
        res[index++] = l[i++];
    }
    while(j < r.length){
        res[index++] = r[j++];
    }
    return res;
}